In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn import cluster, datasets

from matplotlib import pyplot as plt
import random
from collections import Counter


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Toy data with clusters

In [ ]:
blobs = datasets.make_blobs(n_samples=1500, random_state=8)
X,y = blobs

In [ ]:
plt.scatter(X[:, 0], X[:, 1])

In [ ]:
blobs2 = datasets.make_blobs(n_samples=1500, random_state=8, cluster_std=4)
X2,y2 = blobs2
plt.scatter(X2[:, 0], X2[:, 1])

## K-means clustering


In [ ]:
def assign_clusters(X, centroids):
    '''The cluster label for a data point is assigned as simply the 
    centroid that is the closest to it, based on Euclidean distance'''
    
    distances = euclidean_distances(X,centroids)
    labels = distances.argmin(axis=1)
    return labels

def find_centroids(X, labels, centroids):
    '''Centroids are simply the average of all the data in a cluster'''
    
    label_set = set(labels)
    for label in label_set:
        data = X[labels==label]
        centroids[label,:] = data.mean(axis=0)
    return centroids

def kmeans(X, k=3, n_iter=20):

    # We are going to select a random sample of data as our initial 'centroids'

    number_examples = X.shape[0]
    random.seed(42)
    sample = random.sample(range(number_examples),k)
    centroids = X[sample]
    labels = assign_clusters(X,centroids)

    print(centroids)

    # The basic algorithm is to alternately update labels and then centroids
    for iteration in range(n_iter):
        centroids = find_centroids(X, labels, centroids)
        labels = assign_clusters(X,centroids)
        print(f"\n\niteration: {iteration}")
        print(centroids)
    return labels
    

In [ ]:
labels = kmeans(X, k=3, n_iter=0)

In [ ]:
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)

plt.scatter(X[:, 0], X[:, 1], color=colors[labels].tolist())

#### Commentary

Our simple k-means implementation has some challenges, even if it is useful to see how simple the algorithm is. 
* It can, for example, 'lose' clusters if no data points are assigned to a particular cluster.
* It does not check for convergence: if the labels aren't reassigned after an iteration, there is no point to continuing.
* The only initialization method is what is called the _Forgy_ method: choosing random data points. Other options exist, such as randomly assigning the initial clusters (Called _Random Partitioning_) 

But what about the general drawbacks of k-means in particular?

* Like we said before, it assumes that clusters are spheres. What happens if they are elongated or not symmetric?
* You need to know _k_ in advance
* It does not guarantee convergence to the optimal centroids (though is usually workable, in practice)

Let's see what it looks like on data that is not at all spherical


In [ ]:
noisy_moons = datasets.make_moons(n_samples=150, noise=0.05, random_state=8)

X_moons = noisy_moons[0]
plt.scatter(X_moons[:, 0], X_moons[:, 1])

In [ ]:
%time labels_moons = kmeans(X_moons, k=2, n_iter=5) # visually, we know there are 2 clusters

In [ ]:
plt.scatter(X_moons[:, 0], X_moons[:, 1], color=colors[labels_moons].tolist())

Clearly, this is not ideal!  But, this is a general challenge with k-means.  There are, however, other methods that would work better.

Let's take a look at one based on Hierarchical clustering

## Hierarchical clustering


In [ ]:
blobs = datasets.make_blobs(n_samples=150, random_state=8)
X_small,y_small = blobs

In [ ]:
def choose_merge(X, labels):
    '''We merge the 2 clusters that are the closest to each other. We're using a "min" link rule, meaning
    that we use the minimum distance from any pair of points in the cluster. This can lead to some strange
    behavior, though is easy to understand.

    This implementation is really bad, though, because it really only needs to calculate all distances once!
    However, with more general link rules, you may need to recalculate at each step
    '''
    
    label_list = list(set(labels))
    num_labels = len(label_list)

    min_distance = None
    min_pair = None
    
    
    for label_ix1 in range(num_labels-1):
        for label_ix2 in range(label_ix1+1, num_labels):
    

            label1 = label_list[label_ix1]
            label2 = label_list[label_ix2]
            
            data_1 = X[labels==label1]
            data_2 = X[labels==label2]
            
            pair_distances = euclidean_distances(data_1, data_2)
            pair_distance = pair_distances.min()

            if min_distance is None or pair_distance < min_distance:
                min_distance = pair_distance
                min_pair = (label1,label2)

    
    return min_pair
            
    

def agglomerative(X, k=3):
    ''' Typically, you don't specify k here, you run it to completion, then choose the split.
    For this example, it is simpler to show with a fixed k.'''

    labels = np.array(range(X.shape[0]))
    merge_pair = None

    iteration = 0
    while len(set(labels)) > k:
        iteration +=1
        if iteration % 10==0:
            print(f'iteration: {iteration}  Total labels: {len(set(labels))}')
        merge_pair = choose_merge(X, labels)
        labels[labels==merge_pair[1]] = merge_pair[0]
    print(f'iteration: {iteration}  Total labels: {len(set(labels))}')
    return labels


In [ ]:
%time labels_agg = agglomerative(X_small,k=3)

In [ ]:
plt.scatter(X_small[:, 0], X_small[:, 1], color=colors[labels_agg].tolist())

In [ ]:
%time labels_moons_agg = agglomerative(X_moons, k=2) # visually, we know there are 2 clusters

In [ ]:
plt.scatter(X_moons[:, 0], X_moons[:, 1], color=colors[labels_moons_agg].tolist())

### Discussion

Agglomerative clustering like this has the advantage of being based on local neighborhoods of points, so there is no constraint that the clusters need to be spherical.

This can also be a drawback, where two adjacent clusters can get merged together based on some stray points that happen to sit between and 'connect' them.

Also: hierarchical clustering is __Really Slow__.  If, there are N data points, then it is really something like O(N^3) because at each step you compare all pairs (N^2) and you have to do this N times

## Cluster quality

Let's use the Silhouette score to look at whether our clustering is good or not!


In [ ]:

from sklearn.metrics import silhouette_score

k_s = [i for i in range(2,10)]
scores = []

for k in k_s:
    label_k = kmeans(X, k=k, n_iter=10)
    score = silhouette_score(X, label_k)
    scores.append(score)


In [ ]:
plt.plot(k_s, scores)

In [ ]:
k_s = [i for i in range(2,10)]
scores = []

for k in k_s:
    label_k = kmeans(X_moons, k=k, n_iter=10)
    score = silhouette_score(X_moons, label_k)
    scores.append(score)

In [ ]:
plt.plot(k_s, scores)